In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('../../giicg.db')

prompts = pd.read_sql("SELECT * FROM expanded_prompts", conn)
conn.close()
prompts

,message_id,conversation_id,role,message_text,conversational,code,other,gender,user_id,language
0,1,1,user,"parsing data from python iterator, how it coul...","parsing data from python iterator, how it coul...",,,Man (cisgender),6,en
1,730,32,user,Write python function to do operations with in...,Write python function to do operations with in...,,report_dt\tsource\tmetric_name\tmetric_num\tme...,Man (cisgender),6,en
2,1133,55,user,Write shortest tutorial on creating RAG on ema...,Write shortest tutorial on creating RAG on ema...,,,Man (cisgender),6,en
3,1135,55,user,what is FAISS,what is FAISS,,,Man (cisgender),6,en
4,1137,55,user,Transform given code to process large .mbox file,Transform given code to process large .mbox file,,Transform given code to process large .mbox file,Man (cisgender),6,en
...,...,...,...,...,...,...,...,...,...,...
748,1131,54,user,import pandas as pd\nimport numpy as np\nfrom ...,"I want to tune optimal thresholds. Currently, ...",import pandas as pd\nimport numpy as np\nfrom ...,The narratives list looks like this:\nnarrativ...,Man (cisgender),92,en
749,1532,71,user,"from transformers import AutoTokenizer, AutoMo...",I want to use an LLM for listwise reranking in...,"from transformers import AutoTokenizer, AutoMo...",,Man (cisgender),92,en
750,1646,82,user,"def run_query(query, n_results):\n query_em...",this is my code. I want to: Get nodes and edge...,"def run_query(query, n_results):\n query_em...",,Man (cisgender),92,en
751,1849,2,user,\n I am working on the problem of reconstru...,\n I am working on the problem of reconstru...,,Classic CV - Drone navigation\nIf you ever tho...,Man (cisgender),8,en


In [2]:
from helpers.normalization import remove_punctuation_and_newlines, remove_capitalization

prompts['conversational'] = prompts['conversational'].apply(remove_punctuation_and_newlines)
prompts['conversational'] = prompts['conversational'].apply(remove_capitalization)

prompts

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,message_id,conversation_id,role,message_text,conversational,code,other,gender,user_id,language
0,1,1,user,"parsing data from python iterator, how it coul...",parsing data from python iterator how it could...,,,Man (cisgender),6,en
1,730,32,user,Write python function to do operations with in...,write python function to do operations with in...,,report_dt\tsource\tmetric_name\tmetric_num\tme...,Man (cisgender),6,en
2,1133,55,user,Write shortest tutorial on creating RAG on ema...,write shortest tutorial on creating rag on ema...,,,Man (cisgender),6,en
3,1135,55,user,what is FAISS,what is faiss,,,Man (cisgender),6,en
4,1137,55,user,Transform given code to process large .mbox file,transform given code to process large mbox file,,Transform given code to process large .mbox file,Man (cisgender),6,en
...,...,...,...,...,...,...,...,...,...,...
748,1131,54,user,import pandas as pd\nimport numpy as np\nfrom ...,i want to tune optimal thresholds currently i ...,import pandas as pd\nimport numpy as np\nfrom ...,The narratives list looks like this:\nnarrativ...,Man (cisgender),92,en
749,1532,71,user,"from transformers import AutoTokenizer, AutoMo...",i want to use an llm for listwise reranking in...,"from transformers import AutoTokenizer, AutoMo...",,Man (cisgender),92,en
750,1646,82,user,"def run_query(query, n_results):\n query_em...",this is my code i want to get nodes and edges ...,"def run_query(query, n_results):\n query_em...",,Man (cisgender),92,en
751,1849,2,user,\n I am working on the problem of reconstru...,i am working on the problem of reconstruc...,,Classic CV - Drone navigation\nIf you ever tho...,Man (cisgender),8,en


In [3]:
user_prompts = (
    prompts.groupby(['user_id', 'gender'])['conversational']
    .apply(' '.join)
    .reset_index()    # Reset index to create a DataFrame
)

user_prompts.columns = ['user_id', 'gender', 'combined_prompts']
user_prompts = user_prompts[user_prompts['gender'].isin(['Woman (cisgender)', 'Man (cisgender)'])].reset_index()

user_prompts

,index,user_id,gender,combined_prompts
0,0,6,Man (cisgender),parsing data from python iterator how it could...
1,1,8,Man (cisgender),i am working on the problem of reconstruc...
2,2,11,Woman (cisgender),can you adapt the following code so that inste...
3,3,15,Man (cisgender),setalltables action is currently not fetching ...
4,4,16,Woman (cisgender),i want to use dummy hot encoding to replace th...
5,5,25,Man (cisgender),what is the best way to encode and compress a ...
6,6,28,Woman (cisgender),i have a pandas dataframe like this i want to...
7,8,31,Man (cisgender),how can i make use of an observablehqdatabasec...
8,9,34,Man (cisgender),blender and python i have a collection of hund...
9,10,46,Man (cisgender),how to run a python future without blocking ie...


In [51]:
# TF-IDF + Stratified 5-fold CV for Logistic Regression and Linear SVM
# Assumes `user_prompts` DataFrame (with columns 'combined_prompts' and 'gender') is already in the notebook.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, cross_validate, cross_val_predict
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Prepare X, y
X = user_prompts['combined_prompts'].values
# Map labels to binary integers (adjust mapping if you want reverse)
label_map = {'Woman (cisgender)': 0, 'Man (cisgender)': 1}
y = user_prompts['gender'].map(label_map).values

# Stratified 5-fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

# TF-IDF vectorizer settings (tweak as needed)
tfidf_kwargs = dict(
    max_features=500,
    ngram_range=(1, 2),
    #stop_words='english',
    stop_words=['of', 'to', 'is', 'in', 'and', 'that', 'it'],
    #stop_words=['can', 'and', 'you', 'me', 'this', 'that', 'please'],
    min_df=2
)

# Pipelines
pipe_lr = make_pipeline(
    TfidfVectorizer(**tfidf_kwargs),
    LogisticRegression(max_iter=2000, solver='liblinear', random_state=42)
)

pipe_svm = make_pipeline(
    TfidfVectorizer(**tfidf_kwargs),
    LinearSVC(random_state=42)
)

# Cross-validated metrics (fits TF-IDF only on training folds automatically)
cv_results_lr = cross_validate(pipe_lr, X, y, cv=skf, scoring=scoring, n_jobs=-1, return_train_score=False, return_estimator=True)
cv_results_svm = cross_validate(pipe_svm, X, y, cv=skf, scoring=scoring, n_jobs=-1, return_train_score=False, return_estimator=True)

estimators = list(cv_results_lr['estimator']) + list(cv_results_svm['estimator'])
 # list of fitted Pipelines, one per fold

for i, est in enumerate(estimators):
    # find the TfidfVectorizer inside the pipeline robustly
    vec = next(v for v in est.named_steps.values() if isinstance(v, TfidfVectorizer))
    features = vec.get_feature_names_out()
    print(f"\nFold {i}: vocab size = {len(features)}")
    idf = vec.idf_
    top_common = pd.DataFrame({"term": features, "idf": idf}).sort_values("idf").head(10)
    print("Most common (lowest idf) in this fold:")
    print(top_common.to_string(index=False))



def summarize_cv(results, scorer_list=scoring):
    for metric in scorer_list:
        arr = results[f'test_{metric}']
        print(f"{metric:15s} mean = {arr.mean():.4f}   std = {arr.std():.4f}")
    print()


print("Logistic Regression (TF-IDF) CV summary")
summarize_cv(cv_results_lr)

print("Linear SVM (TF-IDF) CV summary")
summarize_cv(cv_results_svm)

# Optional: get cross-validated predictions and a classification report (aggregated)
y_pred_lr = cross_val_predict(pipe_lr, X, y, cv=skf, n_jobs=-1)
y_pred_svm = cross_val_predict(pipe_svm, X, y, cv=skf, n_jobs=-1)

print("Logistic Regression - classification report (aggregated across folds)")
print(classification_report(y, y_pred_lr, target_names=['Woman (cisgender)', 'Man (cisgender)']))
print("Confusion matrix:")
print(confusion_matrix(y, y_pred_lr))

print("\nLinear SVM - classification report (aggregated across folds)")
print(classification_report(y, y_pred_svm, target_names=['Woman (cisgender)', 'Man (cisgender)']))
print("Confusion matrix:")
print(confusion_matrix(y, y_pred_svm))


Fold 0: vocab size = 500
Most common (lowest idf) in this fold:
term      idf
 the 1.000000
 are 1.200671
with 1.200671
 how 1.200671
 can 1.257829
this 1.257829
  an 1.257829
have 1.257829
 not 1.318454
code 1.318454

Fold 1: vocab size = 500
Most common (lowest idf) in this fold:
term      idf
 the 1.000000
 not 1.200671
this 1.257829
 are 1.318454
 can 1.318454
with 1.318454
code 1.318454
 how 1.318454
have 1.318454
 for 1.382992

Fold 2: vocab size = 500
Most common (lowest idf) in this fold:
term      idf
 the 1.000000
code 1.245122
 are 1.302281
this 1.302281
 can 1.302281
 how 1.302281
 not 1.362905
with 1.362905
have 1.362905
  an 1.362905

Fold 3: vocab size = 500
Most common (lowest idf) in this fold:
term      idf
 the 1.000000
this 1.191055
 can 1.245122
 not 1.245122
have 1.245122
 are 1.302281
code 1.302281
 how 1.302281
with 1.302281
  an 1.362905

Fold 4: vocab size = 500
Most common (lowest idf) in this fold:
term      idf
 the 1.000000
this 1.191055
have 1.245122
cod

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [ ]:
# python
# Inspect TF-IDF vocabulary and top terms
# Place this cell after you have fitted your TfidfVectorizer and obtained the TF-IDF matrix.

import numpy as np
import pandas as pd
from scipy import sparse

# --- Adjust these names if your variables differ ---
# Common variable names used in notebooks:
# - fitted vectorizer: "vectorizer", "tfidf_vectorizer", "tfidf"
# - TF-IDF matrix: "X", "X_tfidf", "tfidf_matrix"
vectorizer = globals().get("vectorizer") or globals().get("tfidf_vectorizer") or globals().get("tfidf")
X = globals().get("X") or globals().get("X_tfidf") or globals().get("tfidf_matrix")

# If neither vectorizer nor X exist, try to fit quickly if a docs list is available
if vectorizer is None and "docs" in globals():
    from sklearn.feature_extraction.text import TfidfVectorizer

    vectorizer = TfidfVectorizer(**tfidf_kwargs)
    X = vectorizer.fit_transform(docs)
    print("Fitted a new vectorizer from 'docs' variable.")
elif vectorizer is None:
    raise NameError(
        "Could not find a fitted vectorizer. Define `vectorizer` (or `tfidf_vectorizer`/`tfidf`) or provide `docs`.")

if X is None:
    # try to transform if we have a docs variable
    if "docs" in globals():
        X = vectorizer.transform(docs)
    else:
        raise NameError("Could not find TF-IDF matrix `X` (or `X_tfidf`).")

# --- Basic vocabulary info ---
features = vectorizer.get_feature_names_out()
vocab_size = len(features)
print(f"Vocabulary size (after vectorizer filters): {vocab_size}")

# --- Inspect IDF values ---
idf = np.array(vectorizer.idf_)
idf_df = pd.DataFrame({"term": features, "idf": idf})
idf_df_sorted = idf_df.sort_values("idf")

top_k = 25
print("\nMost common terms (lowest idf):")
print(idf_df_sorted.head(top_k).to_string(index=False))

print("\nRarest terms (highest idf):")
print(idf_df_sorted.tail(top_k).sort_values("idf", ascending=False).to_string(index=False))

# --- Top terms by average TF-IDF across documents ---
if sparse.issparse(X):
    mean_tfidf = np.asarray(X.mean(axis=0)).ravel()
else:
    mean_tfidf = np.array(X).mean(axis=0)

tfidf_df = pd.DataFrame({"term": features, "mean_tfidf": mean_tfidf})
tfidf_top = tfidf_df.sort_values("mean_tfidf", ascending=False).head(top_k)
print(f"\nTop {top_k} terms by mean TF-IDF across the corpus:")
print(tfidf_top.to_string(index=False))


# --- Helper: top terms for one document (by document index) ---
def top_terms_for_doc(doc_idx, top_n=15):
    if doc_idx < 0 or doc_idx >= X.shape[0]:
        raise IndexError("doc_idx out of range")
    row = X[doc_idx]
    if sparse.issparse(row):
        row = row.toarray().ravel()
    else:
        row = np.array(row).ravel()
    nz_idx = np.argsort(row)[::-1][:top_n]
    return pd.DataFrame({"term": features[nz_idx], "tfidf": row[nz_idx]})

# Example usage:
# print(top_terms_for_doc(0, top_n=20))